In [17]:
%pip install -r requirements_part1.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Imports
import pandas as pd

In [12]:
# Load training data
df_all_features = pd.read_csv('liar2/train.csv')

df_all_features.head(3)

,id,label,statement,date,subject,speaker,speaker_description,state_info,true_counts,mostly_true_counts,half_true_counts,mostly_false_counts,false_counts,pants_on_fire_counts,context,justification
0,13847,5,"90 percent of Americans ""support universal bac...","October 2, 2017",government regulation;polls and public opinion...,chris abele,"Chris Abele is Milwaukee County Executive, a p...",wisconsin,1,4,5,3,5,2,a tweet,"""Universal"" is the term for background checks ..."
1,13411,1,Last year was one of the deadliest years ever ...,"May 19, 2017",after the fact;congress;criminal justice;histo...,thom tillis,Thom Tillis is a Republican who serves as U.S....,north carolina,0,2,7,3,2,0,a press release supporting the Back The Blue A...,"Sen. Thom Tillis, a North Carolina Republican,..."
2,10882,0,"Bernie Sanders's plan is ""to raise your taxes ...","October 28, 2015",taxes,chris christie,"Chris Christie announced June 6, 2023 that he ...",national,21,20,27,11,17,8,"Boulder, Colo","Christie said that Sanders’s plan is ""to raise..."


In [14]:
df = df_all_features[["statement", "label"]]
df.head(3)

,statement,label
0,"90 percent of Americans ""support universal bac...",5
1,Last year was one of the deadliest years ever ...,1
2,"Bernie Sanders's plan is ""to raise your taxes ...",0


In [16]:
df["label"].value_counts()

label
1    5284
3    2967
2    2882
4    2743
0    2425
5    2068
Name: count, dtype: int64

| Numeric Label | Text Label       |
|--------------|-----------------|
| 0            | True            |
| 1            | Mostly-true     |
| 2            | Half-true       |
| 3            | Mostly-false    |
| 4            | False           |
| 5            | Pants on Fire   |

# TEST of BERT

In [19]:
from transformers import pipeline

# Load the BERT-based NER model
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# Test with a simple example
text = "Barack Obama was born in Hawaii and worked at the White House."
ner_results = ner_pipeline(text)

# Print detected entities
print(ner_results)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


[{'entity': 'I-PER', 'score': 0.9991485, 'index': 1, 'word': 'Barack', 'start': 0, 'end': 6}, {'entity': 'I-PER', 'score': 0.999493, 'index': 2, 'word': 'Obama', 'start': 7, 'end': 12}, {'entity': 'I-LOC', 'score': 0.9994849, 'index': 6, 'word': 'Hawaii', 'start': 25, 'end': 31}, {'entity': 'I-LOC', 'score': 0.99794334, 'index': 11, 'word': 'White', 'start': 50, 'end': 55}, {'entity': 'I-LOC', 'score': 0.998552, 'index': 12, 'word': 'House', 'start': 56, 'end': 61}]


# REAL

In [25]:
df_sample_all = pd.read_csv('liar2/train_sample.csv')

In [27]:
df_sample = df_sample_all[["statement", "label"]]

df_sample.head(3)


,statement,label
0,"90 percent of Americans ""support universal bac...",5
1,Last year was one of the deadliest years ever ...,1
2,"Bernie Sanders's plan is ""to raise your taxes ...",0


In [28]:
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


In [29]:
def extract_named_entities(text):
    """Runs NER on text and returns a list of extracted entities with their labels."""
    entities = ner_pipeline(text)
    return [(ent["word"], ent["entity"]) for ent in entities]  # Store (word, entity type)

# Apply NER to all statements
df_sample["entities"] = df_sample["statement"].apply(extract_named_entities)

# Show first 5 results
df_sample.head()

/var/folders/7j/vryddfrn7yz83qx1z0rshgph0000gn/T/ipykernel_20241/2247650231.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["entities"] = df_sample["statement"].apply(extract_named_entities)


,statement,label,entities
0,"90 percent of Americans ""support universal bac...",5,"[(Americans, I-MISC)]"
1,Last year was one of the deadliest years ever ...,1,[]
2,"Bernie Sanders's plan is ""to raise your taxes ...",0,"[(Bernie, I-PER), (Sanders, I-PER)]"
